In [182]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import itertools

In [183]:
url = "https://www.fantasynutmeg.com/api/history/season/2016-17"

In [184]:
data = requests.get(url)

In [185]:
hist = data.json()['history']

In [186]:
df = pd.DataFrame(columns=['Name','ID','Position'])

In [187]:
for i in range(len(hist)):
    name = hist[i]['first_name']+" "+hist[i]['second_name']
    id_ = hist[i]['id']
    pos = hist[i]['position']
    df.loc[i] = [name,id_,pos]

In [188]:
players = ["Alexis Sánchez","Gary Cahill","César Azpilicueta","Eden Hazard","Diego Da Silva Costa","Leighton Baines","Harry Kane","Romelu Lukaku","Sergio Agüero","Zlatan Ibrahimovic","Bamidele Alli","Christian Eriksen","Kevin De Bruyne","Kyle Walker","Marcos Alonso"]
df = df[df['Name'].isin(players)]

In [189]:
df

,Name,ID,Position
11,Alexis Sánchez,12,MID
76,César Azpilicueta,77,DEF
77,Gary Cahill,78,DEF
81,Eden Hazard,82,MID
96,Diego Da Silva Costa,97,FWD
124,Leighton Baines,125,DEF
142,Romelu Lukaku,143,FWD
234,Kevin De Bruyne,235,MID
238,Sergio Agüero,239,FWD
271,Zlatan Ibrahimovic,272,FWD


Top 5 players by position for the season(16-17)

Kane, Lukaku, Costa, Aguero, Ibrahimovic  
Sanchez, Alli, Hazard, Erikson, De Bruyne  
Cahill, Alonso, Azpi, Walker, Baines  
Heaton, Lloris, Courtois, de Gea, Cech

Sanchez: https://www.fantasynutmeg.com/api/history/season/2016-17?player=12  
403, 143, 97, 239, 272  
12, 398, 82, 394, 235  
78, 561, 77, 384, 125

In [190]:
player_stats = {}

In [191]:
data = requests.get("https://www.fantasynutmeg.com/api/history/season/2016-17?player=12")

In [192]:
data.json()["history"][0]

{'assists': 0,
 'attempted_passes': 17,
 'big_chances_created': 0,
 'big_chances_missed': 0,
 'bonus': 0,
 'bps': 0,
 'clean_sheets': 0,
 'clearances_blocks_interceptions': 0,
 'completed_passes': 12,
 'creativity': 15.3,
 'dribbles': 1,
 'ea_index': 0,
 'element': 12,
 'errors_leading_to_goal': 0,
 'errors_leading_to_goal_attempt': 0,
 'fixture': 8,
 'fouls': 0,
 'goals_conceded': 4,
 'goals_scored': 0,
 'ict_index': 3.5,
 'id': 12,
 'influence': 5.4,
 'key_passes': 1,
 'kickoff_time': '2016-08-14T15:00:00Z',
 'kickoff_time_formatted': '14 Aug 16:00',
 'loaned_in': 0,
 'loaned_out': 0,
 'minutes': 90,
 'offside': 4,
 'open_play_crosses': 0,
 'opponent_team': 9,
 'opponent_team_name': 'LIV',
 'opponent_team_promoted': False,
 'opponent_team_relegated': False,
 'own_goals': 0,
 'penalties_conceded': 0,
 'penalties_missed': 0,
 'penalties_saved': 0,
 'recoveries': 0,
 'red_cards': 0,
 'round': 1,
 'saves': 0,
 'selected': 296455,
 'tackled': 4,
 'tackles': 0,
 'target_missed': 0,
 'team_

In [193]:
for i in range(len(df)):
    temp_dict = {}
    data = requests.get("https://www.fantasynutmeg.com/api/history/season/2016-17?player="+str(df.iloc[i]["ID"]))
    data = data.json()["history"]
    print(len(data))
    for j,obj in enumerate(data):
        other_dict = {} 
        other_dict["cost"] = obj["value"]
        other_dict["points"] = obj["total_points"]
        temp_dict[obj["round"]] = other_dict
    
    player_stats[df.iloc[i]["Name"]] = temp_dict

38
38
38
38
38
38
38
38
38
38
38
38
38
38
35


In [194]:
player_stats["Marcos Alonso"]

{4: {'cost': 60, 'points': 0},
 5: {'cost': 60, 'points': 0},
 6: {'cost': 59, 'points': 1},
 7: {'cost': 59, 'points': 6},
 8: {'cost': 59, 'points': 6},
 9: {'cost': 59, 'points': 8},
 10: {'cost': 59, 'points': 6},
 11: {'cost': 59, 'points': 13},
 12: {'cost': 61, 'points': 6},
 13: {'cost': 61, 'points': 2},
 14: {'cost': 62, 'points': 6},
 15: {'cost': 63, 'points': 6},
 16: {'cost': 63, 'points': 7},
 17: {'cost': 64, 'points': 6},
 18: {'cost': 64, 'points': 6},
 19: {'cost': 64, 'points': 3},
 20: {'cost': 65, 'points': 1},
 21: {'cost': 65, 'points': 21},
 22: {'cost': 66, 'points': 6},
 23: {'cost': 66, 'points': 2},
 24: {'cost': 66, 'points': 8},
 25: {'cost': 67, 'points': 2},
 26: {'cost': 68, 'points': 2},
 27: {'cost': 68, 'points': 2},
 29: {'cost': 68, 'points': 6},
 30: {'cost': 68, 'points': 1},
 31: {'cost': 68, 'points': 2},
 32: {'cost': 68, 'points': 8},
 33: {'cost': 68, 'points': 0},
 34: {'cost': 68, 'points': 4},
 35: {'cost': 68, 'points': 6},
 36: {'cost'

In [195]:
attackers = []
midfielders = []
defenders = []

for i in range(len(df)):
    if df.iloc[i]["Position"] == "MID":
        midfielders.append(df.iloc[i]["Name"])
    elif df.iloc[i]["Position"] == "FWD":
        attackers.append(df.iloc[i]["Name"])
    else:
        defenders.append(df.iloc[i]["Name"])

In [196]:
attackers

['Diego Da Silva Costa',
 'Romelu Lukaku',
 'Sergio Agüero',
 'Zlatan Ibrahimovic',
 'Harry Kane']

In [197]:
midfielders

['Alexis Sánchez',
 'Eden Hazard',
 'Kevin De Bruyne',
 'Christian Eriksen',
 'Bamidele Alli']

In [198]:
defenders

['César Azpilicueta',
 'Gary Cahill',
 'Leighton Baines',
 'Kyle Walker',
 'Marcos Alonso']

In [199]:
a = [1,2,3,4,5]
m = [1,2,3,4,5]
d = [1,2,3,4,5]

In [200]:
state_space = []
for combination_a in itertools.combinations(a, 3):
    for combination_b in itertools.combinations(m, 3):
        for combination_c in itertools.combinations(d, 3):
            state_space.append({
                'attack': combination_a,
                'midfield': combination_b,
                'defence': combination_c
            })

In [201]:
len(state_space)

1000

In [202]:
state = state_space[420]
state

{'attack': (1, 3, 5), 'midfield': (1, 2, 5), 'defence': (1, 2, 3)}

In [203]:
week = 1

In [204]:
attack_ranking = []
midfield_ranking = []
defense_ranking = []

for i in attackers:
    temp_obj={}
    temp_obj["Name"] = i
    temp_obj["Points"] = player_stats[i][week]["points"]
    temp_obj["Cost"] = player_stats[i][week]["cost"]
    attack_ranking.append(temp_obj)

In [205]:
attack_ranking

[{'Name': 'Diego Da Silva Costa', 'Points': 5, 'Cost': 95},
 {'Name': 'Romelu Lukaku', 'Points': 0, 'Cost': 90},
 {'Name': 'Sergio Agüero', 'Points': 9, 'Cost': 130},
 {'Name': 'Zlatan Ibrahimovic', 'Points': 9, 'Cost': 115},
 {'Name': 'Harry Kane', 'Points': 2, 'Cost': 110}]

In [206]:
attack_ranking = sorted(attack_ranking, key = lambda x : -x["Points"])

In [207]:
attack_ranking

[{'Name': 'Sergio Agüero', 'Points': 9, 'Cost': 130},
 {'Name': 'Zlatan Ibrahimovic', 'Points': 9, 'Cost': 115},
 {'Name': 'Diego Da Silva Costa', 'Points': 5, 'Cost': 95},
 {'Name': 'Harry Kane', 'Points': 2, 'Cost': 110},
 {'Name': 'Romelu Lukaku', 'Points': 0, 'Cost': 90}]

In [208]:
for i in state["attack"]:
    print(i)

1
3
5


In [209]:
def get_state_players(week,state):
    attack_ranking = []
    midfield_ranking = []
    defense_ranking = []

    for i in attackers:
        temp_obj={}
        temp_obj["Name"] = i
        if week in player_stats[i].keys():
            temp_obj["Points"] = player_stats[i][week]["points"]
            temp_obj["Cost"] = player_stats[i][week]["cost"]
        else:
            temp_obj["Points"] = -100
            temp_obj["Cost"] = 200
        attack_ranking.append(temp_obj)
    for i in midfielders:
        temp_obj={}
        temp_obj["Name"] = i
        if week in player_stats[i].keys():
            temp_obj["Points"] = player_stats[i][week]["points"]
            temp_obj["Cost"] = player_stats[i][week]["cost"]
        else:
            temp_obj["Points"] = -100
            temp_obj["Cost"] = 200
        midfield_ranking.append(temp_obj)
    for i in defenders:
        temp_obj={}
        temp_obj["Name"] = i
        if week in player_stats[i].keys():
            temp_obj["Points"] = player_stats[i][week]["points"]
            temp_obj["Cost"] = player_stats[i][week]["cost"]
        else:
            temp_obj["Points"] = -100
            temp_obj["Cost"] = 200
        defense_ranking.append(temp_obj)
        
    attack_ranking = sorted(attack_ranking, key = lambda x : -x["Points"])
    midfield_ranking = sorted(midfield_ranking, key = lambda x : -x["Points"])
    defense_ranking = sorted(defense_ranking, key = lambda x : -x["Points"])
    
    return_obj = {}
    return_att = []
    return_mid = []
    return_def = []
    
    for i in state["attack"]:
        return_att.append(attack_ranking[i-1])
    for i in state["midfield"]:
        return_mid.append(midfield_ranking[i-1])
    for i in state["defence"]:
        return_def.append(defense_ranking[i-1])
    return_obj["attack"] = return_att
    return_obj["midfield"] = return_mid   
    return_obj["defense"] = return_def
    
    return return_obj

In [210]:
get_state_players(week,state)

{'attack': [{'Name': 'Sergio Agüero', 'Points': 9, 'Cost': 130},
  {'Name': 'Diego Da Silva Costa', 'Points': 5, 'Cost': 95},
  {'Name': 'Romelu Lukaku', 'Points': 0, 'Cost': 90}],
 'midfield': [{'Name': 'Eden Hazard', 'Points': 10, 'Cost': 100},
  {'Name': 'Alexis Sánchez', 'Points': 2, 'Cost': 110},
  {'Name': 'Bamidele Alli', 'Points': 2, 'Cost': 85}],
 'defense': [{'Name': 'Kyle Walker', 'Points': 6, 'Cost': 55},
  {'Name': 'César Azpilicueta', 'Points': 5, 'Cost': 60},
  {'Name': 'Gary Cahill', 'Points': 2, 'Cost': 60}]}

In [217]:
get_state_players(30,state)

{'attack': [{'Name': 'Sergio Agüero', 'Points': 7, 'Cost': 127},
  {'Name': 'Diego Da Silva Costa', 'Points': 1, 'Cost': 107},
  {'Name': 'Harry Kane', 'Points': 0, 'Cost': 112}],
 'midfield': [{'Name': 'Bamidele Alli', 'Points': 6, 'Cost': 89},
  {'Name': 'Eden Hazard', 'Points': 5, 'Cost': 102},
  {'Name': 'Alexis Sánchez', 'Points': 2, 'Cost': 116}],
 'defense': [{'Name': 'César Azpilicueta', 'Points': 1, 'Cost': 64},
  {'Name': 'Leighton Baines', 'Points': 1, 'Cost': 57},
  {'Name': 'Marcos Alonso', 'Points': 1, 'Cost': 68}]}